# Lyric and Audio-Based Music Video Generation

### Imports

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import cv2
import os
import argparse
from youtube_transcript_api import YouTubeTranscriptApi
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

### Create Lyric File

In [ ]:
def create_lyric_file(lyric_files_folder, song_name, youtube_video_id):
    transcript = YouTubeTranscriptApi.get_transcript(youtube_video_id)
    lyric_file_save_path = f"{lyric_files_folder}/{song_name}.txt"

    new_file = open(lyric_file_save_path, 'w')
    new_file.write("00:00 " + song_name + "\n")

    for lyric_dict in transcript:
        lyric = lyric_dict["text"].lower()

        lyric = lyric.replace("\n", " ")
        chars_to_remove = ["♪", "(", ")", "[", "]", "{", "}"]
        for char in chars_to_remove:
            lyric = lyric.replace(char, "")
        
        start_time = lyric_dict["start"]
        duration = lyric_dict["duration"]
        end_time = round(start_time + duration, 2)

        end_time_minutes = str(int(end_time/60))
        if len(end_time_minutes) == 1:
            end_time_minutes = "0" + end_time_minutes
        
        end_time_seconds = str(int(end_time - int(end_time_minutes)*60))
        if len(end_time_seconds) == 1:
            end_time_seconds = "0" + end_time_seconds
        
        line_to_write = f"{end_time_minutes}:{end_time_seconds} {lyric}"
        new_file.write(line_to_write + "\n")

    new_file.close()
    
    return lyric_file_save_path

In [ ]:
def map_audio_features_to_image_style(audio_features):
    energy = audio_features[0]['energy']
    energy_text = ['a watercolor painting', 'Van Gogh', 'a photograph', 'Picasso', 'pop art']
    energy_index = int(energy / 0.2)
    if energy_index >= len(energy_text):
        energy_index = len(energy_text) - 1

    valence = audio_features[0]['valence']
    valence_text = ['dark', 'gray', 'neutral', 'pastel', 'neon']
    valence_index = int(valence / 0.2)
    if valence_index >= len(valence_text):
        valence_index = len(valence_text) - 1

    style_string = f'in the style of {energy_text[energy_index]} in {valence_text[valence_index]} colors'
    
    return style_string

In [ ]:
def update_lyrics_with_style_string(lyric_file, audio_features):
    style_string = map_audio_features_to_image_style(audio_features)

    original_file = open(lyric_file, "r")
    new_lines = []
    for line in original_file:
        new_line = f'{line.rstrip()} {style_string}'
        new_lines.append(new_line)

    modified_lyric_file_save_path = lyric_file.split(".")[0] + "_modified.txt"
    with open(modified_lyric_file_save_path, 'w') as new_file:
        for line in new_lines:
            new_file.write(line)
            new_file.write('\n')
    
    return modified_lyric_file_save_path

### Create Music Video

In [ ]:
def create_music_video_without_audio(generated_images_folder, modified_lyric_file_path, music_video_save_path):
    images = [img for img in os.listdir(generated_images_folder) if img.endswith(".png")]
    
    unique_num_to_image_name = {}
    for img_name in images:
        img_split = img_name.split("_", 1)
        unique_num = int(img_split[0])
        unique_num_to_image_name[unique_num] = img_name

    ordered_images = []
    for i in range(len(images)):
        ordered_images.append(unique_num_to_image_name[i])
        
    with open(modified_lyric_file_path) as f:
        lines = f.readlines()

    lyric_time_length = lines[-1].split()[0]
    lyric_time_length_split = lyric_time_length.split(":")
    lyric_time_length_min = int(lyric_time_length_split[0])
    lyric_time_length_sec = int(lyric_time_length_split[1])
    lyric_time_length_secs = lyric_time_length_min*60 + lyric_time_length_sec
    
    desired_fps = len(ordered_images)/lyric_time_length_secs
    
    frame = cv2.imread(os.path.join(generated_images_folder, ordered_images[0]))
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    video = cv2.VideoWriter(music_video_save_path, fourcc, desired_fps, (height, width))

    for image in ordered_images:
        video.write(cv2.imread(os.path.join(generated_images_folder, image)))

    cv2.destroyAllWindows()
    video.release()

In [ ]:
def add_audio_to_music_video(music_video_without_audio_path, audio_file_path, music_video_with_audio_save_path):
    composite = False
    volume_factor = 1.0

    video_clip = VideoFileClip(music_video_without_audio_path)
    audio_clip = AudioFileClip(audio_file_path)
    start = 0
    end = video_clip.end
    audio_clip = audio_clip.volumex(volume_factor)

    audio_clip = audio_clip.subclip(start, end)
    if composite:
        final_audio = CompositeAudioClip([video_clip.audio, audio_clip])
    else:
        final_audio = audio_clip
    final_clip = video_clip.set_audio(final_audio)
    final_clip.write_videofile(music_video_with_audio_save_path)

### Example Song: Levitating by Dua Lipa

In [ ]:
levitating_song_name = "Levitating"
levitating_youtube_video_id = "G2nJPEDc02k"
levitating_spotify_audio_features = [{'danceability': 0.695,
                                      'energy': 0.884,
                                      'key': 6,
                                      'loudness': -2.278,
                                      'mode': 0,
                                      'speechiness': 0.0753,
                                      'acousticness': 0.0561,
                                      'instrumentalness': 0,
                                      'liveness': 0.213,
                                      'valence': 0.914,
                                      'tempo': 103.014,
                                      'type': 'audio_features',
                                      'id': '39LLxExYz6ewLAcYrzQQyP',
                                      'uri': 'spotify:track:39LLxExYz6ewLAcYrzQQyP',
                                      'track_href': 'https://api.spotify.com/v1/tracks/39LLxExYz6ewLAcYrzQQyP',
                                      'analysis_url': 'https://api.spotify.com/v1/audio-analysis/39LLxExYz6ewLAcYrzQQyP',
                                      'duration_ms': 203808,
                                      'time_signature': 4}]

In [ ]:
levitating_lyric_file_path = create_lyric_file("lyric_files", levitating_song_name, levitating_youtube_video_id)

In [ ]:
levitating_modified_lyric_file_path = update_lyrics_with_style_string(levitating_lyric_file_path,
                                                                      levitating_spotify_audio_features)

In [ ]:
!python3 main.py --epochs 40 \
--textfile "lyric_files/Levitating_modified.txt" \
--audiofile "audio_files/Levitating.m4a" \
--savepath "levitating_generated_images"

In [ ]:
levitating_audio_path = "audio_files/Levitating.m4a"
levitating_generated_images_folder = "levitating_generated_images"
levitating_music_video_without_audio_path = "levitating_music_video.avi"
levitating_music_video_with_audio_path = "levitating_music_video.mp4"

In [ ]:
create_music_video_without_audio(levitating_generated_images_folder,
                                 levitating_modified_lyric_file_path,
                                 levitating_music_video_without_audio_path)

In [ ]:
add_audio_to_music_video(levitating_music_video_without_audio_path,
                         levitating_audio_path,
                         levitating_music_video_with_audio_path)